<a href="https://colab.research.google.com/github/menosgrande/yahAuct-AVpricechecker/blob/main/yahAuct_AVpricechecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 検索ワード入力
print('入力してください')
search_word = input().replace(" ","%20").replace("　","%E3%80%80")
# URLの指定        
url = 'https://erocate.com/search1/q-' + search_word + '/s-ya/c-ya_2084045794/?o=t1&vmode=1'
print(url)
#ユーザーエージェントの設定（設定必須）
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
# ここでcookieを指定
cookie = {'adultauthed': '1617782919'}  #クッキーの指定
#htmlの取得
response = requests.get(url=url, headers=headers, cookies=cookie)
html = response.text
#BeautifulSoupで扱えるようにパースします
soup = BeautifulSoup(html,'lxml')
#URLタイトル
url_title = soup.title.string
print(url_title)
#全体平均
mean_price = soup.find(id = 'mean_price').text
mean_price_word_a = "平均:"
mean_price_word_b = "円"
mean_price_word = mean_price_word_a + str(mean_price) + mean_price_word_b
print(mean_price_word)


In [ ]:
#右カラムから値段と作品タイトル取得
right_content = soup.find(id = 'right_contents')
#作品タイトルリスト
item_title_list = []
for item_title in right_content.find_all(class_ = 'item_title'):
   item_title_strip = item_title.text.replace("\u3000"," ").strip()
   item_title_list.append(item_title_strip)

#作品値段リスト
item_price_list = []
for item_price in right_content.find_all(class_ = "item_price"):
   item_price_strip = item_price.text.replace("\u3000"," ").strip()
   item_price_list.append(item_price_strip)

In [ ]:
#商品リスト
test_list = [];
for i, p in enumerate(item_price_list):
    test_list.append({
        "価格": p,
        "タイトル": item_title_list[i],
    });

#表にする
df = pd.DataFrame(data = test_list)
css = {"background-color": "green"}
display(df.style.set_properties(subset="価格", **css))

In [ ]:
#平均値　LINE送信  
LINE_url = "https://notify-api.line.me/api/notify"
api_token = "wzq0M8MdrUrAqqGAcF5GYVDMdy1hM632xnOkaKQnfL4"
headers = {"Authorization": "Bearer " + api_token}

message = search_word +'は' + mean_price_word + '\n' + url
payload = {'message': message}
r = requests.post(LINE_url, headers=headers, params=payload)